In [1]:
# Importando Pandas e delimitando caracteres p/ coluna
import pandas as pd
pd.set_option('max_colwidth', 200)

# Data
from datetime import datetime

# Geolocalização
import requests
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [3]:
def lat_long(endereco):
    url = 'https://nominatim.openstreetmap.org/search'
    # params = {'q': endereco, 'format': 'json', 'limit': 1, 
    #           'address': {
    #                         'country': 'Brazil',
    #                         'country_code': 'br',
    #                         'state': 'São Paulo'
    #                         }}
    params = {'q': endereco, 'format': 'json', 'limit': 1}

    try: 
        response = requests.get(url, params=params).json()

        if response:
            location = response[0]
            latitude = location['lat']
            longitude = location['lon']
            return f'{latitude},{longitude}'
        else:
            return '0,0'
    except: 
        return '-'

df = pd.read_csv("../dremio_trust.csv")
df.head(2)

,Link_Apto,Endereco,Bairro,Valor_RS,Andares,Tamanho_m2,Valor_RS/m2,Quartos,Suites,Garagem,Banheiros,Varanda,Mobiliado,Portaria,Metro,id,Refdate
0,https://loft.com.br/imovel/apartamento-17-de-janeiro-vila-andrade-sao-paulo-2-quartos-40m2/52nta6,Condomínio Click Morumbi - Rua 17 de Janeiro - Vila Andrade - São Paulo - SP,Vila Andrade,272906,4º andar em diante,40,6822,2,0,1,1,0,Não,24h,100000000,12431,2023-04-17T00:00
1,https://loft.com.br/imovel/apartamento-academia-de-sao-paulo-itaim-paulista-sao-paulo-2-quartos-52m2/19w2z6t,Avenida Academia de São Paulo - Itaim Paulista - São Paulo - SP,Itaim Paulista,245518,3º andar,52,4721,2,1,0,1,0,Não,24h,100000000,12432,2023-04-17T00:00


In [4]:
for i in tqdm(range(df.head(10).shape[0])): 
    try: 
        end = df.loc[i, 'Endereco']

        df.loc[i, 'Lat'] = lat_long(" - ".join(z for z in end.split(' - ')[-3:])).split(',')[0]
        df.loc[i, 'Long'] = lat_long(" - ".join(z for z in end.split(' - ')[-3:])).split(',')[1]
    except Exception as e: 
        print('Indice: {}\nEndereço: {}\nMensagem de erro: {}\n\n'.format(i, end, e))

100%|██████████| 10/10 [00:28<00:00,  2.84s/it]


In [12]:
if (df.query("Lat == '0'").shape[0]) == 0: 
    df.to_csv("../Power BI/Geoloc.csv", sep=';', index=False)

df = pd.read_csv('../Power BI/Geoloc.csv', sep=';')
df.head(2)

,Link_Apto,Endereco,Bairro,Valor_RS,Andares,Tamanho_m2,Valor_RS/m2,Quartos,Suites,Garagem,Banheiros,Varanda,Mobiliado,Portaria,Metro,id,Refdate,Lat,Long
0,https://loft.com.br/imovel/apartamento-17-de-janeiro-vila-andrade-sao-paulo-2-quartos-40m2/52nta6,Condomínio Click Morumbi - Rua 17 de Janeiro - Vila Andrade - São Paulo - SP,Vila Andrade,272906,4º andar em diante,40,6822,2,0,1,1,0,Não,24h,100000000,12431,2023-04-17T00:00,-20.341310,-49.197098
1,https://loft.com.br/imovel/apartamento-academia-de-sao-paulo-itaim-paulista-sao-paulo-2-quartos-52m2/19w2z6t,Avenida Academia de São Paulo - Itaim Paulista - São Paulo - SP,Itaim Paulista,245518,3º andar,52,4721,2,1,0,1,0,Não,24h,100000000,12432,2023-04-17T00:00,-23.501765,-46.399609
